In [39]:

import matplotlib.pyplot as plt
import mediapipe as mp
import os
import cv2
from google.colab.patches import cv2_imshow
from google.colab import files
import numpy  as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
# Path to the directory containing .mat files on Google Drive
mat_files_dir = '/content/drive/MyDrive/AFLW2000'
combined_data = []

# Check if the directory exists
if not os.path.isdir(mat_files_dir):
    raise FileNotFoundError(f"The directory {mat_files_dir} does not exist or cannot be accessed.")

for filename in os.listdir(mat_files_dir):
    if filename.endswith(".mat"):
        # Load the .mat file
        file_path = os.path.join(mat_files_dir, filename)
        mat_file = scipy.io.loadmat(file_path)

        # Replace 'Pose_Para' with the actual name of the variable in the .mat file
        if 'Pose_Para' in mat_file:
            row_vector = mat_file['Pose_Para']  # Extract the row vector

            # Convert the row vector to a DataFrame and append it to the combined_data list
            df = pd.DataFrame(row_vector)
            combined_data.append(df)
        else:
            print(f"'Pose_Para' not found in {filename}")

# Concatenate all DataFrames to create a single dataset
final_dataset = pd.concat(combined_data, ignore_index=True)

# Save the combined dataset to a CSV file in Google Drive
output_path = '/content/drive/MyDrive/combined_dataset.csv'
final_dataset.to_csv(output_path, index=False)

print(final_dataset)

             0         1         2           3           4           5  \
0    -0.251103 -1.024545  0.230088  135.287994  210.972794  -46.908741   
1     0.162750 -0.192906 -0.010402  209.237137  179.394638  -93.713112   
2     0.092695 -0.527315  0.163928  154.975708  182.917969 -109.305893   
3     0.175137 -1.213063 -0.160148  120.905075  181.013519  -23.546370   
4    -0.252779  1.069315 -0.194186  335.404999  207.491760  -30.646679   
...        ...       ...       ...         ...         ...         ...   
1738  0.688345  1.207175  0.414514  327.262543  159.763580  -19.350164   
1739 -1.014593 -2.051753  0.896962  123.145042  139.085663  -10.110525   
1740 -0.128817  0.865954  0.125393  305.163940  200.363907  -53.872231   
1741  0.018578  0.151739  0.000894  250.115433  188.768616 -114.450081   
1742  0.601243  0.081812  0.126460  237.640182  119.955353 -112.205368   

             6  
0     0.001098  
1     0.001133  
2     0.001653  
3     0.001238  
4     0.001218  
...      

In [ ]:
final_dataset=final_dataset.drop([3,4,5,6],axis=1)

In [47]:

final_dataset.columns = ['Yaw', 'Pitch', 'Roll']
final_dataset

ValueError: Length mismatch: Expected axis has 7 elements, new values have 3 elements

In [ ]:
# Initialize mediapipe face mesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

image = cv2.imread('image.jpg')

# convert image from BGR to RGB for mediapipe
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# initialize the facemesh model
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=False, # don't add iris landmarks
    min_detection_confidence=0.5
) as face_mesh:
  # process the image and detect the face landmarks
  results = face_mesh.process(rgb_image)

# draw only the landmarks (points) on the image
if results.multi_face_landmarks:
  for face_landmarks in results.multi_face_landmarks:

    for idx, landmark in enumerate(face_landmarks.landmark):

      # get the position of the landmark
      h, w, _ = image.shape
      x, y = int(landmark.x * w), int(landmark.y * h)


      # draw a small circle at the landmark point
      cv2.circle(image, (x, y), 5, (0, 0, 255), -1)


# display the image with only facial landmarks points
cv2_imshow(image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [24]:
# convert landmarks to DataFrame
df = pd.DataFrame(all_landmarks, columns=['x', 'y', 'Roll', 'Pitch', 'Yaw'])

# prepare the data for the model
X = df[['x', 'y']] # features: landmarks (x, y)
y_roll = df[['Roll']] # target: Roll
y_pitch = df[['Pitch']] # target: Pitch
y_yaw = df[['Yaw']] # target: Yaw

# splitting data into training and testing sets
X_train, X_test, y_train_roll, y_test_roll = train_test_split(X, y_roll, test_size=0.2, random_state=42)
_, _, y_train_pitch, y_test_pitch = train_test_split(X, y_pitch, test_size=0.2, random_state=42)
_, _, y_train_yaw, y_test_yaw = train_test_split(X, y_yaw, test_size=0.2, random_state=42)

# train LinearRegression model for Roll, Pitch, and Yaw
lr_roll = LinearRegression().fit(X_train, y_train_roll)
lr_pitch = LinearRegression().fit(X_train, y_train_pitch)
lr_yaw = LinearRegression().fit(X_train, y_train_yaw)

# making predictions
y_pred_roll = lr_roll.predict(X_test)
y_pred_pitch = lr_pitch.predict(X_test)
y_pred_yaw = lr_yaw.predict(X_test)

# comparing predictions to actual values
results_roll = pd.DataFrame({
    'x':X_test['x'].values,
    'y':X_test['y'].values,
    'Actual Roll':y_test_roll.values.ravel(),
    'Predected Roll':y_pred_roll.ravel()
})

results_pitch = pd.DataFrame({
    'x': X_test['x'].values,
    'y': X_test['y'].values,
    'Actual Pitch': y_test_pitch.values.ravel(),
    'Predicted Pitch': y_pred_pitch.ravel()
})

results_yaw = pd.DataFrame({
    'x': X_test['x'].values,
    'y': X_test['y'].values,
    'Actual Yaw': y_test_yaw.values.ravel(),
    'Predicted Yaw': y_pred_yaw.ravel()
})

print('Predected vs Actual (Roll):')
print(results_roll)

print('Predected vs Actual (Pitch):')
print(results_pitch)

print('Predected vs Actual (Yaw):')
print(results_yaw)



Predected vs Actual (Roll):
       x     y  Actual Roll  Predected Roll
0   1400  1357     1.527525        1.527525
1   1015  1274     1.527525        1.527525
2   1063  1465     1.527525        1.527525
3   1868   996     1.527525        1.527525
4   1450  2137     1.527525        1.527525
..   ...   ...          ...             ...
89  1598  1457     1.527525        1.527525
90  1925  1273     1.527525        1.527525
91   819  1623     1.527525        1.527525
92  1472  1620     1.527525        1.527525
93  1789  1389     1.527525        1.527525

[94 rows x 4 columns]
Predected vs Actual (Pitch):
       x     y  Actual Pitch  Predicted Pitch
0   1400  1357     89.986506        89.986506
1   1015  1274     89.986506        89.986506
2   1063  1465     89.986506        89.986506
3   1868   996     89.986506        89.986506
4   1450  2137     89.986506        89.986506
..   ...   ...           ...              ...
89  1598  1457     89.986506        89.986506
90  1925  1273     89.98